In [2]:
import pandas as pd
staff = pd.DataFrame([{'name':'Sally','role':'Director of HR'},
                     {'name':'Kelly','role':'Course liasion'},
                     {'name':'Jack','role':'Grader'}])

staff = staff.set_index('name')

student = pd.DataFrame([{'name':'James','school':'Business'},
                     {'name':'Kelly','school':'Engineering'},
                     {'name':'Jack','school':'Law'}])

student = student.set_index('name')

print(staff.head())
print(student.head())

                 role
name                 
Sally  Director of HR
Kelly  Course liasion
Jack           Grader
            school
name              
James     Business
Kelly  Engineering
Jack           Law


In [3]:
#If we want the Union of these, we would call merge passing in the DataFrame on the left and the DataFrame on the right and 
#telling merge that we want it to use an outer join.We would want to use the left and right indexes as the joining columns.

pd.merge(staff, student, how = 'outer', left_index= True, right_index = True)
#We see in the resulting DataFrame that everybody is listed and since Mike does not have a role 
#and John does not have a school, those cells are listed as missing values.

,role,school
name,,
Jack,Grader,Law
James,NaN,Business
Kelly,Course liasion,Engineering
Sally,Director of HR,NaN


In [4]:
#If we wanted to get the intersection that is just those people who are a student and a staff member, 
#we can set the how attribute to enter.

pd.merge(staff, student, how = 'inner', left_index= True, right_index = True)

,role,school
name,,
Kelly,Course liasion,Engineering
Jack,Grader,Law


Now, there are two other common use cases when merging DataFrames, and both are examples of what we would call set addition. The first is when we want to get a list of all staff regardless of whether they're students or not. But if there were students, we'd want to get their student details as well. To do this, we would use a left join. It's important to note that the order of DataFrames in this function, the first DataFrame is the left DataFrame and the second is the right.

In [5]:
pd.merge(staff, student, how = 'left', left_index= True, right_index = True)

,role,school
name,,
Sally,Director of HR,NaN
Kelly,Course liasion,Engineering
Jack,Grader,Law


In [6]:
pd.merge(staff, student, how = 'right', left_index= True, right_index = True)

,role,school
name,,
James,NaN,Business
Kelly,Course liasion,Engineering
Jack,Grader,Law


We can also do it one other way. The merge method has a couple of other interesting parameters. First, you don't need to use indices to join on, you can use columns as well. Here's an example. Here we have a parameter called "on" and then we can assign a column that both DataFrames, how? Using the joining column

In [7]:
staff = staff.reset_index()
student = student.reset_index()

pd.merge(staff, student, how = 'right', on = 'name')

,name,role,school
0,Kelly,Course liasion,Engineering
1,Jack,Grader,Law
2,James,NaN,Business


In [8]:
# So what happens when we have conflicts between the DataFrames? Let's take a look by creating new staff and
# student DataFrames that have a location information added to them.
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 
                          'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 
                          'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 
                          'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 
                            'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 
                            'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 
                            'Location': '512 Wilson Crescent'}])

# In the staff DataFrame, this is an office location where we can find the staff person. And we can see the
# Director of HR is on State Street, while the two students are on Washington Avenue, and these locations just
# happen to be right outside my window as I film this. But for the student DataFrame, the location information
# is actually their home address.

# The merge function preserves this information, but appends an _x or _y to help differentiate between which
# index went with which column of data. The _x is always the left DataFrame information, and the _y is always
# the right DataFrame information.

# Here, if we want all the staff information regardless of whether they were students or not. But if they were
# students, we would want to get their student details as well.Then we can do a left join and on the column of
# Name

pd.merge(staff_df, student_df, how='left', on='Name')

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


In [9]:
# From the output, we can see there are columns Location_x and Location_y. Location_x refers to the Location
# column in the left dataframe, which is staff dataframe and Location_y refers to the Location column in the
# right dataframe, which is student dataframe.

# Before we leave merging of DataFrames, let's talk about multi-indexing and multiple columns. It's quite
# possible that the first name for students and staff might overlap, but the last name might not. In this
# case, we use a list of the multiple columns that should be used to join keys from both dataframes on the on
# parameter. Recall that the column name(s) assigned to the on parameter needs to exist in both dataframes.

# Here's an example with some new student and staff data
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])

# As you see here, James Wilde and James Hammond don't match on both keys since they have different last
# names. So we would expect that an inner join doesn't include these individuals in the output, and only Sally
# Brooks will be retained.
pd.merge(staff_df, student_df, how='inner', on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


In [10]:
# If we think of merging as joining "horizontally", meaning we join on similar values in a column found in two
# dataframes then concatenating is joining "vertically", meaning we put dataframes on top or at the bottom of
# each other

# Let's understand this from an example. You have a dataset that tracks some information over the years. And
# each year's record is a separate CSV and every CSV ofr every year's record has the exactly same columns.
# What happens if you want to put all the data, from all years' record, together? You can concatenate them.

In [11]:
%%capture
df_2011 = pd.read_csv("datasets/college_scorecard/MERGED2011_12_PP.csv", error_bad_lines=False)
df_2012 = pd.read_csv("datasets/college_scorecard/MERGED2012_13_PP.csv", error_bad_lines=False)
df_2013 = pd.read_csv("datasets/college_scorecard/MERGED2013_14_PP.csv", error_bad_lines=False)

FileNotFoundError: [Errno 2] File b'datasets/college_scorecard/MERGED2011_12_PP.csv' does not exist: b'datasets/college_scorecard/MERGED2011_12_PP.csv'